# EDA - Exploratory Data Analysis

Exploratory Data Analysis refers to the critical process of performing initial investigations on data so as to discover patterns,to spot anomalies,to test hypothesis and to check assumptions with the help of summary statistics and graphical representations (Patil 2018).

## Goals

- **Data cleaning:** cleaning the database, outliers, missings;
- **Data Investigation:** describe the customer base on purchase;


### Import Librarys

In [1]:
# Data Analysis Step
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
from scipy import stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# plot parameters
%matplotlib inline
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['figure.facecolor'] = '#00000000'

# outliers with mahalanobis
from scipy.spatial import distance



## Data Preparation


### Import DataSet

In [2]:
# dataset from the Excel file 
dataset = pd.read_excel('../data/online_retail_II.xlsx')


In [3]:
df = dataset.copy()

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
def summary(df):
    """
    Display a summary of a DataFrame.

    This function displays the first few rows of the DataFrame, basic information about the DataFrame's structure,
    and summary statistics including percentiles.

    Parameters:
    df (DataFrame): The pandas DataFrame to be summarized.

    Returns:
    None
    """

    display(f'Rows: {df.shape[0]}; Columns: {df.shape[1]}')
    display(df.head())
    print('-' * 100)
    display(df.info())
    print('-' * 100)
    display(df.describe([0.01, 0.25, 0.50, 0.75, 0.99]))

In [ ]:
summary(df)

### Normalize Date

In [5]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [6]:
# change columns to lowercase and remove whitespace in between 
df.columns = df.columns.str.replace(" ", "_").str.lower()

In [7]:
df.columns

Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country'],
      dtype='object')

## Data Cleaning

### Understanding the data

In [8]:
# Unique Values 
print(f'invoice_unique: {df.invoice.unique().shape}')
print(f'stockcode_unique: {df.stockcode.unique().shape}')
print(f'description_unique: {df.description.unique().shape}')
print(f'quantity_unique: {df.quantity.unique().shape}')

invoice_unique: (28816,)
stockcode_unique: (4632,)
description_unique: (4682,)
quantity_unique: (825,)


In [9]:
# Start/End of date
df.invoicedate.min(), df.invoicedate.max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2010-12-09 20:01:00'))

##### Handling missing values

In [10]:
# Is there null valeus?
df.isnull().sum()

invoice             0
stockcode           0
description      2928
quantity            0
invoicedate         0
price               0
customer_id    107927
country             0
dtype: int64

In [11]:
df = df.dropna(subset=['customer_id', 'description'])
df.isnull().sum()

invoice        0
stockcode      0
description    0
quantity       0
invoicedate    0
price          0
customer_id    0
country        0
dtype: int64

### Remove negative values

- negative values are returned items;
- `invoice` that contai 'C' are returned values;

In [12]:
df.describe([0.01, 0.05, 0.10, 0.20, 0.90, 0.95, 0.99]).T

,count,mean,min,1%,5%,10%,20%,50%,90%,95%,99%,max,std
quantity,417534.0,12.758815,-9360.0,-2.0,1.0,1.0,1.0,4.0,24.0,36.0,144.0,19152.0,101.220424
invoicedate,417534,2010-07-01 01:12:19.787609856,2009-12-01 07:45:00,2009-12-02 15:04:00,2009-12-13 12:42:00,2010-01-18 14:49:00,2010-03-05 09:09:00,2010-07-08 19:12:00,2010-11-17 17:16:00,2010-11-28 13:43:00,2010-12-07 14:41:00,2010-12-09 20:01:00,NaN
price,417534.0,3.887547,0.0,0.29,0.42,0.65,0.85,1.95,6.75,8.5,14.95,25111.09,71.131797
customer_id,417534.0,15360.645478,12346.0,12435.0,12725.0,13042.0,13624.0,15311.0,17706.0,17913.0,18196.0,18287.0,1680.811316


In [13]:
df_sorted = df.sort_values(by='quantity')
df_sorted.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277.0,France
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277.0,France
359630,C524235,16047,POP ART PEN CASE & PENS,-5184,2010-09-28 11:02:00,0.08,14277.0,France
359636,C524235,37340,MULTICOLOUR SPRING FLOWER MUG,-4992,2010-09-28 11:02:00,0.10,14277.0,France


In [14]:
df_filtered = df[~df['invoice'].astype(str).str.startswith('C')]
df_filtered.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
df = df_filtered.copy()

In [16]:
df.describe([0.01, 0.05, 0.10, 0.20, 0.90, 0.95, 0.99]).T

,count,mean,min,1%,5%,10%,20%,50%,90%,95%,99%,max,std
quantity,407695.0,13.586686,1.0,1.0,1.0,1.0,2.0,5.0,24.0,36.0,144.0,19152.0,96.842229
invoicedate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2009-12-02 15:10:00,2009-12-13 13:27:00,2010-01-19 11:01:00,2010-03-05 11:31:00,2010-07-09 15:46:00,2010-11-18 10:37:00,2010-11-28 13:53:00,2010-12-07 14:31:00,2010-12-09 20:01:00,NaN
price,407695.0,3.294188,0.0,0.29,0.42,0.65,0.85,1.95,6.75,8.5,14.95,10953.5,34.756655
customer_id,407695.0,15368.504107,12346.0,12435.0,12731.0,13044.0,13635.0,15321.0,17706.0,17913.0,18196.0,18287.0,1679.7957


### Check Duplicates

The pandas `df.duplicated()` function uses a comparison of all columns by default to identify duplicate rows in a DataFrame. If you do not specify specific columns, it checks whether all columns have the same values ​​in different rows.

In [17]:
# Getting the number of rows in the dataframe
df.shape[0]

407695

In [18]:
# Displaying the number of duplicate rows
print(f"The dataset contains {df.duplicated().sum()} duplicate rows that need to be removed.")

# Removing duplicate rows
df.drop_duplicates(inplace=True)

The dataset contains 6748 duplicate rows that need to be removed.


In [19]:
# Getting the number of rows in the dataframe
df.shape[0]

400947

#### Export DataSets

## Data Investigation - EDA

### let's create the sales column

In [20]:
df['sales'] = df['quantity'] * df['price']

In [21]:
df.sales.quantile([0, 0.25, 0.5, 0.75, 1])

0.00        0.0
0.25        5.0
0.50       12.5
0.75       19.5
1.00    15818.4
Name: sales, dtype: float64

In [22]:
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


**5 - Customer Profile Based on Purchasing Frequency**

In [23]:
# new df group by customer ID
customer_groups = df.groupby('customer_id')

# customer profile using agregate function
customer_profile = customer_groups.agg({
    'invoice': 'count',       # Number of orders
    'sales': 'sum',            # Total sales
    'quantity': 'sum',         # Total quantity purchased
    'invoicedate': 'mean'    # Purchased date
})

# rename the columns 
customer_profile = customer_profile.rename(columns={
    'invoice': 'Frequency',
    'sales': 'Total Sales',
    'quantity': 'Total Quantity Purchased',
    'invoicedate': 'Purchased date'
})

# sort the customer profile by ppurchasing frequency
customer_profile = customer_profile.sort_values(by='Frequency', ascending=False)

# reset the index for a cleaner output
customer_profile.reset_index(inplace=True)

customer_profile

,customer_id,Frequency,Total Sales,Total Quantity Purchased,Purchased date
0,14911.0,5568,152121.22,69709,2010-07-09 10:35:50.237069056
1,17841.0,4944,29562.02,14332,2010-06-08 08:53:10.485436928
2,14606.0,3802,18482.10,9287,2010-05-28 10:34:06.075749632
3,14156.0,2646,196549.74,108105,2010-06-10 04:30:19.659864064
4,12748.0,2532,22457.90,13110,2010-08-09 21:21:22.109004800
...,...,...,...,...,...
4309,13906.0,1,42.00,100,2010-10-26 15:00:00.000000000
4310,14328.0,1,445.05,1,2010-09-27 16:01:00.000000000
4311,15893.0,1,305.28,288,2010-08-15 11:12:00.000000000
4312,13942.0,1,320.00,1000,2010-06-23 12:35:00.000000000


## Export DataSet

In [24]:
df.to_csv('../output/online_retail_ead.csv', index=False) 

## Bibliography

-  P. Patil, "What is Exploratory Data Analysis?," Medium, Mar. 2018. [Online]. Available: https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15.[Accessed: Octuber 23, 2023].
- D. Andrés, "Clean your Time Series data III: Outliers removal - ML Pills," Apr. 2023. [Online]. Available: https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15. [Accessed: Octuber 23, 2023].
- [Customer Segmentation & Recommendation System](https://www.kaggle.com/code/farzadnekouei/customer-segmentation-recommendation-system#Step-3.1-%7C-Handling-Missing-Values)
- [RFM Analysis Online Retail II](https://www.kaggle.com/code/ekrembayar/rfm-analysis-online-retail-ii)




 









 